In [ ]:
from openai import OpenAI

client = OpenAI(
    base_url="https://api.groq.com/openai/v1",  # or your provider for gpt-oss-120b
    api_key="__YOUR_API_KEY__",  # replace with your actual API key
)

N = 5
k = 3

system_prompt = """
You are a helpful assistant. Solve this puzzle for me.
You can represent actors with a_1, a_2, ... and agents with A_1, A_2, ... . Your solution
must be a list of boat moves where each move indicates the people on the boat. For example, if
there were two actors and two agents, you should return:

moves=[["A_2", "a_2"], ["A_2"], ["A_1", "A_2"], ["A_1"], ["A_1", "a_1"]]

which indicates that in the first move, A_2 and a_2 row from left to right, and in the second
move, A_2 rows from right to left and so on.

Requirements:
• When exploring potential solutions in your thinking process, always include the
  corresponding complete list of boat moves.
• The list shouldn’t have comments.
• Ensure your final answer also includes the complete list of moves for final solution.
"""

user_prompt = f"""
You will write python program to solve the below problem.
You will only write code blocks.
Your python program must be executable and returns the right answer for the problem.
Q: {N} actors and their {N} agents want to cross a river in a boat that is capable of holding
only {k} people at a time, with the constraint that no actor can be in the presence
of another agent, including while riding the boat, unless their own agent is also
present, because each agent is worried their rivals will poach their client. Initially, all actors
and agents are on the left side of the river with the boat. How should they cross the river?
(Note: the boat cannot travel empty)

# solution using Python:
"""

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt},
]

In [ ]:
def verify_moves(
    moves: list[tuple[str, ...]],
    boat_capacity: int | None = None,
    verbose: bool = False,
) -> None:
    """Raise ValueError on the first rule violation; otherwise return None."""
    def is_safe(group: set[str]) -> bool:
        actors  = {p for p in group if p.startswith("a")}
        agents  = {p for p in group if p.startswith("A")}
        for act in actors:
            g = f"A{act[1:]}"          # guardian name
            if g not in agents and any(other != g for other in agents):
                return False
        return True

    people = {p.strip() for step in moves for p in step}
    n_pairs = len(people) // 2
    if boat_capacity is None:
        boat_capacity = 2 if n_pairs <= 3 else 3

    left, right = set(people), set()
    boat_left   = True               # boat starts on the left

    for t, step in enumerate(moves, start=1):
        step = tuple(p.strip() for p in step)

        if verbose:
            side = "L" if boat_left else "R"
            print(f"\nStep {t}  (boat on {side}, carrying {step})")
            print(f"  left : {sorted(left)}")
            print(f"  right: {sorted(right)}")

        # ---- rule checks ---------------------------------------------------
        if not step:
            raise ValueError(f"Step {t}: boat cannot travel empty")
        if len(step) > boat_capacity:
            raise ValueError(f"Step {t}: exceeds boat capacity {boat_capacity}")

        current, other = (left, right) if boat_left else (right, left)
        if any(p not in current for p in step):
            missing = [p for p in step if p not in current]
            raise ValueError(f"Step {t}: {missing} not on the current bank")

        if not is_safe(set(step)):
            raise ValueError(f"Step {t}: unsafe mix *inside the boat* {step}")
        if not is_safe(current - set(step)):
            raise ValueError(f"Step {t}: unsafe mix on the departing bank after leaving")
        if not is_safe(other):
            raise ValueError(f"Step {t}: unsafe mix on the destination bank before arrival")

        # ---- perform the crossing -----------------------------------------
        current -= set(step)
        other   |= set(step)
        boat_left = not boat_left        # toggle side

        if not is_safe(other):
            raise ValueError(f"Step {t}: unsafe mix on the destination bank after arrival")

    if left:
        raise ValueError("Finished, but some individuals are still on the left bank")


    # If we made it here, everything is valid
    return  # success → no exception

In [ ]:
import importlib
import traceback
import uuid
import os

N_RUNS = 5
results = []

for run in range(1, N_RUNS + 1):
    print(f"\n========== RUN {run} ==========")

    # 1. Call the model
    response = client.chat.completions.create(
        model="openai/gpt-oss-120b",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.0,
        max_tokens=4096,
    )

    code_output = response.choices[0].message.content
    print("\nRaw model output:")
    print(code_output)

    # 2. Clean the code (remove fences)
    clean_code = (
        code_output
        .replace("```python", "")
        .replace("```", "")
        .strip()
    )

    # 3. Save to a uniquely named file
    module_name = f"rc_solution_{uuid.uuid4().hex}"
    file_path = f"{module_name}.py"

    with open(file_path, "w") as f:
        f.write(clean_code)

    try:
        # 4. Import the module
        module = importlib.import_module(module_name)

        # Reload to ensure clean import
        importlib.reload(module)

        # 5. Run the solver
        moves = module.solve()
        print("\nMoves from solver:")
        print(moves)

        # 6. Verify the solution
        verify_moves(moves, boat_capacity=k, verbose=False)

        print("✔ PASS — Valid Solution Found")
        results.append(True)

    except Exception as e:
        print("✘ FAIL — Error occurred")
        traceback.print_exc()
        results.append(False)

    # 7. Cleanup temporary module
    os.remove(file_path)

print("\n========== SUMMARY ==========")
print(f"Total Runs: {N_RUNS}")
print(f"Successes:  {sum(results)}")
print(f"Failures:   {N_RUNS - sum(results)}")
print(f"Success Rate: {sum(results) / N_RUNS * 100:.1f}%")